![pres.png](pres.PNG)

##  <div style="color:#9E1A1A">L'intégration numérique </div>

Les méthodes d’intégration numérique sont en général utilisées soit pour intégrer une fonction analytique mais dont on ne peut pas connaître la primitive, soit pour intégrer une fonction connue uniquement sous forme discrète (issue de mesures expérimentales par exemple). Les techniques d’intégration numérique sont souvent basées sur la propriété d’égalité entre la valeur de l’intégrale et l’aire (signée) sous la courbe. 
![intg.png](intg.PNG)

In [14]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
from ipywidgets import *

##  <div style="color:#9E1A1A">La méthode de point milieu : </div>

In [15]:
class Milieu(object): 
    def __init__(self, a, b, n, f):
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)
        self.f = f
        self.n = n
    def integrate(self,f):
        x=self.x
        h = float(x[1] - x[0])
        s=0
        for i in range(self.n):
            s=s+f((x[i]+x[i+1])*0.5)
        return h*s
       
    def Graph(self,f,resolution=1001):
        xl = self.x
        yl=f(xl);
        xlist_fine=np.linspace(self.a, self.b, resolution)        
        for i in range(self.n):            
            m=(xl[i]+xl[i+1])/2
            x_rect = [xl[i], xl[i], xl[i+1], xl[i+1], xl[i]] 
            y_rect = [0  , f(m), f(m)  , 0 , 0 ] 
            plt.plot(x_rect, y_rect,"r")
            plt.plot(m,f(m),"g*")
            
        yflist_fine = f(xlist_fine)
        plt.plot(xlist_fine, yflist_fine,'b')
        plt.xlabel('x')
        plt.ylabel('f(x)')
        plt.title('Milieu')
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

##  <div style="color:#9E1A1A">Le schéma de Simpson:</div>

La méthode de Simpson consiste à remplacer la courbe  $y = f(x)$ par un arc de parabole sur l’intervalle$ [x_i-_1,x_i+_1]$ , puis à intégrer la fonction ainsi obtenue sur l’intégrale .
![arc.png](arc.PNG)
L’arc de parabole passant par les trois points , on calcule son équation et l’intégrer facilement sur l’intervalle $[x_i-_1,x_i+_1]$
<br> Puisque les intégrales prennent en compte deux intervalles à la fois, il faudra obligatoirement que <strong>le nombre total d’intervalle n soit pair.</strong>

In [16]:
class Simpson(object):
    def __init__(self, a, b, n, f): 
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)
        self.f = f
        self.n = n #

    def integrate(self,f):
        x=self.x 
        y=f(x)
        h = float(x[1] - x[0])
        n = len(x) - 1
        if n % 2 == 1:
            n =n-1
        s = y[0] + y[n] + 4.0 * sum(y[1:-1:2]) + 2.0 * sum(y[2:-2:2])
       
        return h * s / 3.0
    def Graph(self,f,resolution=1001):
        xl = self.x 
        yl = f(xl) 
        xlist_fine=np.linspace(self.a, self.b, resolution)
       
        for i in range(self.n):
            x1=np.linspace(xl[i], xl[i+1], resolution)
            m=(xl[i]+xl[i+1])/2
            bg=xl[i]
            bd=xl[i+1]
            l0 = (x1-m)/(bg-m)*(x1-bd)/(bg-bd)
            l1 = (x1-bg)/(m-bg)*(x1-bd)/(m-bd)
            l2 = (x1-bg)/(bd-bg)*(x1-m)/(bd-m)
            P = f(bg)*l0 + f(m)*l1 + f(bd)*l2
            plt.plot(x1,P,'b')
            plt.plot(m,f(m),"g*")
        yflist_fine = f(xlist_fine)
        plt.plot(xlist_fine, yflist_fine,'b')
        plt.plot(xl, yl,'r')
        
        plt.ylabel('f(x)')
        plt.title('Simpson')
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )
       

##  <div style="color:#9E1A1A">La méthode des rectangles: </div>

Considérons donc une fonction de $R$ dans $R$ continue sur un intervalle $[a,b]$. Pour un physicien, intégrer signifie la plupart du temps calculer l'aire sous la courbe de la fonction entre a et b. 
La première méthode qui vienne à l'esprit, c'est de découper l'aire entre la courbe $f(x)$, l'axe des x et les droites$ x= a $et $x = b$, en une multitude de petits rectangles. Découpons l'intervalle $[a,b]$ en rectangles élémentaires de largeur h, h étant petit. Le rectangle n° i aura donc pour longueur $f(a + i*h)$. Sa surface est donc égale à $h*f(a + i*h)$. L'aire sous la courbe entre a et b est obtenue en sommant tous ces petits rectangles. Reste qu'en posant cette relation, j'ai fait l'hypothèse implicite que la courbe limite le coté gauche de mon rectangle. On peut imaginer d'autres découpages. Voyons cela sur un schéma.

In [17]:
class RectangleG ( object ) :
    def __init__ (self , a , b , n , f ) :
        self.a = a
        self.b = b
        self.x = np.linspace( a , b , n+1 )
        self.f = f
        self.n = n
    def integrate ( self , f ) :
        x= self.x
        y= f( x )
        h = float( x[1] - x[0] )
        s = sum( y[ 0 : -1 ] )
        return h * s
    def Graph ( self , f , resolution =1001 ) :
        xl = self.x
        yl = f(xl)
        xlist_fine =np.linspace( self.a , self.b , resolution )
        for i in range ( self.n ) :
            x_rect = [xl[ i ] , xl[ i ] , xl[ i + 1 ] , xl[i+1] , xl[ i ] ] # abscisses des sommets
            y_rect = [0 , yl[ i ] , yl[ i ] , 0 , 0 ] # ordonnees des sommets
            plt.plot ( x_rect , y_rect , 'r' )
        yflist_fine = f ( xlist_fine )
        plt.plot ( xlist_fine , yflist_fine )
        plt.plot(xl, yl,"bo")
        plt.xlabel ( 'x' )
        plt.ylabel ( ' f ( x ) ' )
        plt.title ( ' Methode des rectangles gauches' )
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

##  <div style="color:#9E1A1A">La méthode des trapèzes: </div>

On considère une fonction $f(x)$ définie sur un intervalle $[a,b]$  , et dont on veut calculer l’intégrale $ ∫ f(x) dx$ sur ce même intervalle. La méthode des trapèzes consiste à diviser l’intervalle $[a,b]$  en une série d’intervalles $[x_i-_1,x_i]$   , à remplacer la courbe $y = f(x)$ par un segment de droite, et à calculer l’aire du trapèze ainsi obtenu.
![trapeze.png](trapeze.PNG)

In [18]:
class Trapezoidal(object):
    def __init__(self, a, b, n, f):
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)
        self.f = f
        self.n = n
    def integrate(self,f):
        x=self.x
        y=f(x)
        h = float(x[1] - x[0])
        s = y[0] + y[-1] + 2.0*sum(y[1:-1])
        return h * s / 2.0
    def Graph(self,f,resolution=1001):
        xl = self.x
        yl = f(xl)
        xlist_fine=np.linspace(self.a, self.b, resolution)
        for i in range(self.n):
            x_rect = [xl[i], xl[i], xl[i+1], xl[i+1], xl[i]] # abscisses des sommets
            y_rect = [0   , yl[i], yl[i+1]  , 0     , 0   ] # ordonnees des sommets
            plt.plot(x_rect, y_rect,"m")
        yflist_fine = f(xlist_fine)
        plt.plot(xlist_fine, yflist_fine)#plot de f(x)
        plt.plot(xl, yl,"cs")#point support
        plt.ylabel('f(x)')
        plt.title ( ' Methode des trapèzes' )

        plt.text(0.2*(self.a+self.b), 0.25*f(self.b), '$I_{%s} =\int_{a}^{b}f(x)dx$=%12.4f' % (self.n,self.integrate(f)), fontsize=15)

##  <div style="color:#9E1A1A"> Simulation </div>

In [19]:
def simule(n,f,a,b):
    
    R=RectangleG(a,b,n,f)
    T=Trapezoidal(a,b,n,f)
    M=Milieu(a,b,n,f)
    S=Simpson(a,b,n,f)
    plt.subplot(321)
    R.Graph(f)  
    plt.subplot(322)    
    T.Graph(f) 
    plt.subplot(325)
    M.Graph(f)
    plt.subplot(326)
    S.Graph(f)    
    plt.show()

In [20]:
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [21]:
output = widgets.Output()
with output:
    fig= plt.figure(figsize=(10,7))
fig.canvas.toolbar_position = 'bottom'

<Figure size 720x504 with 0 Axes>

In [22]:
int_slider = widgets.IntSlider(value=2, min=1, max=50, step=1, description='Choisir N')
select = widgets.Dropdown(options={'1/(1+x**2)':lambda x:1/(1+x**2),
                                   'x**2-x+5':lambda x:x**2-x+5,
                                   'x**2':lambda x:x**2,
                                   'x**3':lambda x:x**3,
                                   'tang(x)':lambda x:tan(x),
                                   'arctang(x)':lambda x:arctan(x),
                                    'sin(x)':lambda x: sin(x),
                                    'cos(x)':lambda x:cos(x)},description='Choisir une fonction')      
text_a = widgets.IntText(value=-1,description='Valeur de a', continuous_update=False)
text_b = widgets.IntText(value=1,description='Valeur de b', continuous_update=False)
button = widgets.Button(description="Afficher")

In [23]:
def on_button_clicked(b):
    with output:
        fig.clear()
        simule(int_slider.value,select.value,text_a.value,text_b.value)

##  <div style="color:#9E1A1A"> Choisir valeur N , Fonction et a et b :  </div>

In [25]:
vbox1 = widgets.VBox([int_slider,select,text_a,text_b,button])
vbox2 = widgets.VBox([ output])

box_layout = widgets.Layout()

vbox1.layout = box_layout
vbox2.layout = box_layout
button.on_click(on_button_clicked)
widgets.HBox([vbox1, vbox2])